<a href="https://colab.research.google.com/github/angirov/pubmed_crawler/blob/main/save_bydate_saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 --version
!pip install Bio

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [32]:
import os

dir = "/content/drive/MyDrive/dsr/pubmed_data/"
os.chdir(dir)
os.getcwd()

'/content/drive/MyDrive/dsr/pubmed_data'

In [33]:
import os
from tqdm import tqdm
import pandas as pd
from Bio import Entrez
Entrez.email = "example@example.com"
from urllib.error import HTTPError
from pathlib import Path

In [37]:
def get_pmids_from_one_day(topic, date):
    handle = Entrez.esearch(db="pubmed", 
                            term= topic + "[MeSH Terms]", 
                            retmax=10000,
                            datetype="pdat", 
                            mindate=date, 
                            maxdate=date, )
    record = Entrez.read(handle)
    ids = record["IdList"]
    return ids


class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
# print(color.BOLD + 'Hello, World!' + color.END)


def print_err(text):
    err_text = color.BOLD + color.RED + text + color.END
    print(err_text)
    return err_text


def get_pmids_from_period(topic, span):
    handle = Entrez.esearch(db="pubmed", 
                            term= topic + "[MeSH Terms]",
                            retmax=1000,
                            datetype="pdat",
                            mindate=span[0],
                            maxdate=span[1], )
    record = Entrez.read(handle)
    ids = record["IdList"]
    return ids


import time
def get_xml(id):
    attempt = 1
    while attempt <= 5:
        try:
            handle = Entrez.efetch(db="pubmed", 
                            id=id, 
                            retmode="xml", 
                            rettype="abstract")
            xml = handle.read()
            return xml
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print_err(f">>>>> Attempt {attempt}: Received error from server {err}")
                attempt += 1
                time.sleep(15)
            else:
                attempt += 1
                time.sleep(15)
    print_err(f">>>>> Failed to download {id}. Error: {err}")
    return ""

    # pp = pprint.PrettyPrinter(indent=4)
    # pp.pprint(json.dumps(xmltodict.parse(xml)))


def get_xmls_ids(ids):
    pairs = []
    for id in tqdm(ids):
        xml = get_xml(id)
        pairs += [(id, xml)]
    return pairs


def save_pair(csv_writer, id, xml):
    row = id, xml
    csv_writer.writerow(row)
    # print(f"Saved paper {id}.")


def save_xmls_ids(file_name, date, ids):
    with open(file_name,'a+') as out:
        csv_writer=csv.writer(out, dialect='unix')
        for id in tqdm(ids):
            xml = get_xml(id)
            save_pair(csv_writer, id, xml)
    print(f"Saved {len(ids)} papers published on {date}.")
    return file_name


import csv
def save_span(span, list_of_pairs):
    file_name = span[0].replace("/", "_") + "__" + span[1].replace("/", "_") + ".csv"
    with open(file_name,'a+') as out:
        csv_out=csv.writer(out, dialect='unix')
        # csv_out.writerow(['name','num'])
        # for row in list_of_pairs:
        #     csv_out.writerow(row)
        csv_out.writerows(list_of_pairs)
    return file_name


def save_date(date, list_of_pairs):
    file_name = date2filename(date)
    with open(file_name,'a+') as out:
        csv_out=csv.writer(out, dialect='unix')
        # csv_out.writerow(['name','num'])
        # for row in list_of_pairs:
        #     csv_out.writerow(row)
        csv_out.writerows(list_of_pairs)
        # print(f"Saved {len(list_of_pairs)} papers published on {date}.")
    return file_name


def ymd2date(year, month, day):
    return f'{year}/{month:02}/{day:02}'


def date2ymd(date):
    year, month, day = str.split(date, sep="/")
    return year, month, day


def date2filename(dir, date):
    year, month, _ = date2ymd(date)
    filename = year + "/" + month + "/" + date.replace("/", "_") + ".csv"
    print(f"File name: {filename}")
    return filename


In [53]:
year = 2022
month = 11


for day in range(28, 32):
    date = ymd2date(year, month, day)

    ids = get_pmids_from_one_day("breast cancer", date)
    print(f"Number of papers published on  {date}': {len(ids)}")
    if len(ids) == 0:
        break

    file_name = Path(date2filename(dir, date))
    path = Path(dir + "/" + f"{year}" + "/" + f"{month:03}" + "/")
    path.mkdir(parents=True, exist_ok=True)
    
    print(ids)
    print("=================")

    if file_name.absolute().is_file():
        print(f"File {file_name} already exists.")
        df = pd.read_csv(file_name, header=None, names=['id', 'xml'])
        ids_done = df.id.astype(int).astype(str).to_list()
        print("DONE:")
        print(ids_done)
        print("=================")
        ids = [id for id in ids if id not in ids_done]
        print(f"Number of papers already saved': {len(ids_done)}. To save: {len(ids)}")

    print(ids)
    save_xmls_ids(file_name, date, ids)


Number of papers published on  2022/11/28': 35
File name: 2022/11/2022_11_28.csv
['36638314', '36621399', '36518248', '36500393', '36500383', '36499224', '36497066', '36479042', '36476677', '36464602', '36463702', '36462440', '36456385', '36455802', '36455743', '36455683', '36455505', '36455483', '36455408', '36450322', '36450315', '36449787', '36449765', '36449762', '36443878', '36443711', '36443538', '36443508', '36443319', '36441723', '36441271', '36437424', '36397195', '36347024', '36310371']
File 2022/11/2022_11_28.csv already exists.
DONE:
['36455483', '36455408', '36450322', '36450315', '36449787', '36449765', '36449762', '36443878', '36443711', '36443538']
Number of papers already saved': 10. To save: 25
['36638314', '36621399', '36518248', '36500393', '36500383', '36499224', '36497066', '36479042', '36476677', '36464602', '36463702', '36462440', '36456385', '36455802', '36455743', '36455683', '36455505', '36443508', '36443319', '36441723', '36441271', '36437424', '36397195', '

100%|██████████| 25/25 [00:09<00:00,  2.69it/s]


Saved 25 papers published on 2022/11/28.
Number of papers published on  2022/11/29': 19
File name: 2022/11/2022_11_29.csv
['36730355', '36730330', '36730147', '36730128', '36531979', '36524129', '36499265', '36497078', '36495783', '36470073', '36463642', '36455759', '36446827', '36446780', '36446754', '36446608', '36446386', '36446052', '36445561']
File 2022/11/2022_11_29.csv already exists.
DONE:
['36730355', '36730330', '36730147', '36730128', '36531979', '36524129', '36499265', '36497078', '36495783', '36470073', '36463642', '36455759', '36446827', '36446780', '36446754', '36446608', '36446386', '36446052', '36445561']
Number of papers already saved': 19. To save: 0
[]


0it [00:00, ?it/s]

Saved 0 papers published on 2022/11/29.


Number of papers published on  2022/11/30': 29
File name: 2022/11/2022_11_30.csv
['36657935', '36547151', '36547150', '36547148', '36506936', '36506911', '36481356', '36465484', '36462594', '36462463', '36462381', '36460099', '36451297', '36451243', '36451239', '36451108', '36451034', '36450882', '36449636', '36449489', '36448998', '36448918', '36447408', '36408915', '36385461', '36346290', '36130642', '36084874', '36049568']
File 2022/11/2022_11_30.csv already exists.
DONE:
['36657935', '36547151', '36547150', '36547148', '36506936', '36506911', '36481356', '36465484', '36462594', '36462463', '36462381', '36460099', '36451297', '36451243', '36451239', '36451108', '36451034', '36450882', '36449636', '36449489', '36448998', '36448918', '36447408', '36408915', '36385461', '36346290', '36130642', '36084874', '36049568']
Number of papers already saved': 29. To save: 0
[]


0it [00:00, ?it/s]

Saved 0 papers published on 2022/11/30.


Number of papers published on  2022/11/31': 0


In [39]:
year = 2022

for month in range(10, 13):
    for day in range(1, 32):
        date = ymd2date(year, month, day)

        ids = get_pmids_from_one_day("breast cancer", date)
        print(f"Number of papers published on  {date}': {len(ids)}")
        if len(ids) == 0:
            break

        file_name = Path(date2filename(dir, date))
        path = Path(dir + "/" + f'{year}' + "/" + f"{month:02}" + "/")
        path.mkdir(parents=True, exist_ok=True)

        if file_name.is_file():
            df = pd.read_csv(file_name, header=None, names=['id', 'xml'])
            ids_done = df.id.to_list()
            ids = [id for id in ids if id not in ids_done]
            print(f"Number of papers already saved': {len(ids_done)}. To save: {len(ids)}")

        save_xmls_ids(file_name, date, ids)


Number of papers published on  2022/10/01': 756
File name: 2022/10/2022_10_01.csv


100%|██████████| 756/756 [06:23<00:00,  1.97it/s]


Saved 756 papers published on 2022/10/01.
Number of papers published on  2022/10/02': 7
File name: 2022/10/2022_10_02.csv


100%|██████████| 7/7 [00:02<00:00,  2.77it/s]


Saved 7 papers published on 2022/10/02.
Number of papers published on  2022/10/03': 40
File name: 2022/10/2022_10_03.csv


...

100%|██████████| 8/8 [00:18<00:00,  2.32s/it]


Saved 8 papers published on 2022/11/27.
Number of papers published on  2022/11/28': 35
File name: 2022/11/2022_11_28.csv


100%|██████████| 35/35 [00:13<00:00,  2.58it/s]


Saved 35 papers published on 2022/11/28.


HTTPError: ignored

In [ ]:
# from pathlib import Path

# year = "2022"
# month = '04'
# day = '01'

# date = ymd2date(year, month, day)
# file_name = Path(date2filename)
# if file_name.is_file():
#     df = pd.read_csv(file_name, header=None, names=['id', 'xml'])
#     ids_done = df.id.to_list()
#     ids = 
